## Sample Script for 1 df
### In this notebook we unfold the steps of the transformation pipeline

In [1]:
import os
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd()
if PROJECT_DIR.stem == 'data':
    PROJECT_DIR = PROJECT_DIR.parents[1]
    sys.path.insert(0, PROJECT_DIR.as_posix())
    os.chdir(PROJECT_DIR.as_posix())
    %load_ext autoreload
    %autoreload 2

In [2]:
# TODO: find out if indexing improves performance or not

## First, in polars

In [3]:
import os
from pathlib import Path
from functools import reduce
from dataclasses import dataclass
from typing import Dict, List
import logging

import numpy as np
import pandas as pd
import polars as pl

from src.data.process_data import load_participant_datasets
from src.data.config_data import DataConfigBase
from src.data.config_data_imotions import iMotionsConfig, IMOTIONS_LIST
from src.data.config_data_raw import RawConfig, RAW_LIST
from src.data.config_data_trial import TrialConfig, TRIAL_LIST
from src.data.config_participant import ParticipantConfig, PARTICIPANT_LIST

from src.log_config import configure_logging
configure_logging()


In [4]:
list_of_data_configs = [
    IMOTIONS_LIST,
    # RAW_LIST,
    # TRIAL_LIST,
]

participant_data = load_participant_datasets(PARTICIPANT_LIST[0], IMOTIONS_LIST)
participant_data.trial, participant_data.eda

(shape: (12, 2)
 ┌─────────────┬──────────────┐
 │ Timestamp   ┆ Stimuli_Seed │
 │ ---         ┆ ---          │
 │ f64         ┆ i64          │
 ╞═════════════╪══════════════╡
 │ 188041.074  ┆ 1            │
 │ 468085.8797 ┆ 1            │
 │ 549706.8485 ┆ 2            │
 │ 829753.3666 ┆ 2            │
 │ …           ┆ …            │
 │ 1.6999e6    ┆ 5            │
 │ 1.9799e6    ┆ 5            │
 │ 2.0235e6    ┆ 6            │
 │ 2.3036e6    ┆ 6            │
 └─────────────┴──────────────┘,
 shape: (307_539, 4)
 ┌───────────┬───────────────┬──────────┬───────────────────────────┐
 │ Timestamp ┆ EDA_d_Battery ┆ EDA_RAW  ┆ EDA_d_PacketReceptionRate │
 │ ---       ┆ ---           ┆ ---      ┆ ---                       │
 │ f64       ┆ f64           ┆ f64      ┆ i64                       │
 ╞═══════════╪═══════════════╪══════════╪═══════════════════════════╡
 │ 50.8622   ┆ 3648.307692   ┆ 7.343876 ┆ 100                       │
 │ 50.8622   ┆ 3625.005128   ┆ 7.33877  ┆ 100                 

### before we can anything we need to merge trial information

In [5]:
trial = participant_data.trial
temperature = participant_data.temperature

new = trial.join(
    trial,
    on='Timestamp',
    how='outer_coalesce',
    # join_nulls=True,
).sort(
    'Timestamp'
)
new

Timestamp,Stimuli_Seed,Stimuli_Seed_right
f64,i64,i64
188041.074,1,1
468085.8797,1,1
549706.8485,2,2
829753.3666,2,2
899309.6762,3,3
1.1793e6,3,3
1.2966e6,4,4
1.5767e6,4,4
1.6999e6,5,5


In [6]:
# Special case for imotions data: we need to add the stimuli seed column
# datasets are missing the trial information (via Stimuli_Seed) and need to be merged with the trial data first
for data_config in IMOTIONS_LIST:
    # add the stimuli seed column to all datasets of the participant
    if "Stimuli_Seed" not in participant_data.datasets[data_config.name].dataset.columns:
        participant_data.datasets[data_config.name].dataset = participant_data.datasets[data_config.name].dataset.join(
            participant_data.trial,
            on='Timestamp',
            how='outer_coalesce',
        ).sort(
            'Timestamp'
        )
    assert participant_data.datasets[data_config.name].dataset['Timestamp'].is_sorted(descending=False)


participant_data.eda

Timestamp,EDA_d_Battery,EDA_RAW,EDA_d_PacketReceptionRate,Stimuli_Seed
f64,f64,f64,i64,i64
50.8622,3648.307692,7.343876,100,null
50.8622,3625.005128,7.33877,100,null
51.8618,3654.133333,7.33877,100,null
76.7951,3638.112821,7.333665,100,null
78.7875,3642.482051,7.333665,100,null
81.7816,3665.784615,7.333665,100,null
83.7741,3658.502564,7.333665,100,null
87.7637,3639.569231,7.32856,100,null
87.7637,3648.307692,7.32856,100,null


### add trial information

In [7]:

def create_trial_index(df):
    """Create a trial index based on the stimuli seed which is originally send once at the start and end of each trial."""
    # Check if the trial index already exists
    if 'Trial' in df.index.names:
        return df
    # 0. Check if all trials are complete  # TODO
    # 1. Forward fill and backward fill columns
    ffill = df['Stimuli_Seed'].ffill()
    bfill = df['Stimuli_Seed'].bfill()
    # 2. Where forward fill and backward fill are equal, replace the NaNs in the original Stimuli_Seed
    df['Stimuli_Seed'] = np.where(ffill == bfill, ffill, df['Stimuli_Seed'])
    # 3. Only keep rows where the Stimuli_Seed is not NaN
    df = df[df['Stimuli_Seed'].notna()]
    df['Stimuli_Seed'] = df['Stimuli_Seed'].astype(int)
    # Create a new column that contains the trial number
    df['Trial'] = df.Stimuli_Seed.diff().ne(0).cumsum()
    # Add Trial to the index
    df.set_index('Trial', append=True if 'Time' in df.index.names else False, inplace=True)
    return df

def create_trial_column(df):
    """Create a trial index based on the stimuli seed which is originally send once at the start and end of each trial."""
    # Check if the trial index already exists # TODO

    # 0. Check if all trials are complete  # TODO
    # 1. Forward fill and backward fill columns
    ffill = df['Stimuli_Seed'].forward_fill()
    bfill = df['Stimuli_Seed'].backward_fill()
    # 2. Where forward fill and backward fill are equal, replace the NaNs in the original Stimuli_Seed
    df['Stimuli_Seed'] = np.where(ffill == bfill, ffill, df['Stimuli_Seed'])
    # 3. Only keep rows where the Stimuli_Seed is not NaN
    df = df[df['Stimuli_Seed'].notna()]
    df['Stimuli_Seed'] = df['Stimuli_Seed'].astype(int)
    # Create a new column that contains the trial number
    df['Trial'] = df.Stimuli_Seed.diff().ne(0).cumsum()
    # Add Trial to the index
    df.set_index('Trial', append=True if 'Time' in df.index.names else False, inplace=True)
    return df


In [29]:
# Forward and backward fill
ffill = participant_data.eda['Stimuli_Seed'].fill_null(strategy='forward')
bfill = participant_data.eda['Stimuli_Seed'].fill_null(strategy='backward')
# this is the same as np.where(ffill == bfill, ffill, df['Stimuli_Seed'])
participant_data.eda = participant_data.eda.with_columns(
    pl.when(ffill == bfill)
    .then(ffill)
    .otherwise(participant_data.eda['Stimuli_Seed'])
    .alias('Stimuli_Seed')
)
assert participant_data.datasets[data_config.name].dataset['Timestamp'].is_sorted(descending=False)
# Only keep rows where the Stimuli_Seed is not NaN
participant_data.eda = participant_data.eda.filter(participant_data.eda['Stimuli_Seed'].is_not_null())
# Create a new column that contains the trial number
participant_data.eda['Trial'] = participant_data.eda['Stimuli_Seed'].diff().ne(0).cumsum()

participant_data.eda

/var/folders/54/0yy_ylvj7tx247wkq0n07dnr0000gn/T/ipykernel_86377/4254766233.py:15: DeprecationWarning: `cumsum` is deprecated. It has been renamed to `cum_sum`.
  participant_data.eda['Trial'] = participant_data.eda['Stimuli_Seed'].diff().ne(0).cumsum()


TypeError: DataFrame object does not support `Series` assignment by index

Use `DataFrame.with_columns`.

In [22]:
participant_data.eda

Timestamp,EDA_d_Battery,EDA_RAW,EDA_d_PacketReceptionRate,Stimuli_Seed
f64,f64,f64,i64,i64
50.8622,3648.307692,7.343876,100,null
50.8622,3625.005128,7.33877,100,null
51.8618,3654.133333,7.33877,100,null
76.7951,3638.112821,7.333665,100,null
78.7875,3642.482051,7.333665,100,null
81.7816,3665.784615,7.333665,100,null
83.7741,3658.502564,7.333665,100,null
87.7637,3639.569231,7.32856,100,null
87.7637,3648.307692,7.32856,100,null


In [ ]:
create_trial_column(participant_data.rating)


AttributeError: 'Series' object has no attribute 'forward_fill'

In [ ]:
for data_config in IMOTIONS_LIST:
    if "Stimuli_Seed" not in participant_data.datasets[data_config.name].dataset.columns:
        participant_data.datasets[data_config.name].dataset = participant_data.datasets[data_config.name].dataset.join(
            participant_data.trial,
            on='Timestamp',
            how='outer_coalesce',
        ).sort(
            'Timestamp'
        )

In [ ]:
import pandas as pd
pd.timedelta_range(start='1 days', periods=1000, freq='1s')

TimedeltaIndex(['1 days 00:00:00', '1 days 00:00:01', '1 days 00:00:02',
                '1 days 00:00:03', '1 days 00:00:04', '1 days 00:00:05',
                '1 days 00:00:06', '1 days 00:00:07', '1 days 00:00:08',
                '1 days 00:00:09',
                ...
                '1 days 00:16:30', '1 days 00:16:31', '1 days 00:16:32',
                '1 days 00:16:33', '1 days 00:16:34', '1 days 00:16:35',
                '1 days 00:16:36', '1 days 00:16:37', '1 days 00:16:38',
                '1 days 00:16:39'],
               dtype='timedelta64[ns]', length=1000, freq='S')

In [ ]:
import polars as pl
import datetime
import polars as pl
import numpy as np

In [ ]:
import os
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd()
if PROJECT_DIR.stem == 'data':
    PROJECT_DIR = PROJECT_DIR.parents[1]
    sys.path.insert(0, PROJECT_DIR.as_posix())
    os.chdir(PROJECT_DIR.as_posix())
    %load_ext autoreload
    %autoreload 2

In [ ]:
import os
from pathlib import Path
from functools import reduce
from dataclasses import dataclass
from typing import Dict, List
import logging

import pandas as pd
import polars as pl

from src.data.process_data import load_participant_datasets
from src.data.config_data import DataConfigBase
from src.data.config_data_imotions import iMotionsConfig, IMOTIONS_LIST
from src.data.config_data_raw import RawConfig, RAW_LIST
from src.data.config_data_trial import TrialConfig, TRIAL_LIST
from src.data.config_participant import ParticipantConfig, PARTICIPANT_LIST

from src.log_config import configure_logging
configure_logging()


In [ ]:
@dataclass
class Data:
    """Dataclass for a single csv files"""
    name: str
    dataset: pd.DataFrame

@dataclass
class Participant:
    """Dataclass for a single participant"""
    id: str
    datasets: Dict[str, Data]
    
    def __getattr__(self, name):
        if name in self.datasets:
           return self.datasets[name].dataset
        raise AttributeError(f"'{self.__class__.__name__}' object has no attribute '{name}'")


In [ ]:
# Prepare data configs
TEST_DIR = Path.cwd() / 'data' / '_test'
for data_config in IMOTIONS_LIST:
    data_config.load_dir = TEST_DIR / 'imotions'
    data_config.save_dir = TEST_DIR / 'raw'
for data_config in RAW_LIST:
    data_config.load_dir = TEST_DIR / 'raw'
    data_config.save_dir = TEST_DIR / 'trial'
for data_config in TRIAL_LIST:
    data_config.load_dir = TEST_DIR / 'trial'
    data_config.save_dir = TEST_DIR / 'processed'

participant = PARTICIPANT_LIST[0]

## 1. iMotions -> Raw

### 1.1 iMotions load

In [ ]:
datasets = {}
for data_config in IMOTIONS_LIST:

    # Special case for iMotions data: find the start of the data in the file
    file_path = data_config.load_dir / participant.id / f"{data_config.name_imotions}.csv"
    with open(file_path, 'r') as file:
        lines = file.readlines(2**16) # only read a few lines
        file_start_index = next(i for i, line in enumerate(lines) if "#DATA" in line) + 1

    # Load and process data
    dataset = pd.read_csv(
        file_path,
        skiprows=None if not file_start_index else file_start_index,
        usecols=lambda column: column in data_config.load_columns,
        )

    # Special case for iMotions data: we also want to rename some columns
    if isinstance(data_config, iMotionsConfig):
        dataset.rename(columns=data_config.rename_columns, inplace=True) if data_config.rename_columns else None
    dataset = Data(name=data_config.name, dataset=dataset)
    datasets[data_config.name] = dataset

participant_imotions = Participant(id=participant.id, datasets=datasets)
participant_imotions.eeg


,Timestamp,EEG_RAW_Ch1,EEG_RAW_Ch2,EEG_RAW_Ch3,EEG_RAW_Ch4,EEG_RAW_Ch5,EEG_RAW_Ch6,EEG_RAW_Ch7,EEG_RAW_Ch8
0,5.982810e+01,-4361.342773,-6395.005859,12318.420898,-2381.229004,1936.005981,6634.855957,-3910.541992,-2543.639893
1,6.009020e+01,-4367.398926,-6375.790039,12322.760742,-2373.503906,1938.581055,6633.663086,-3906.250000,-2539.920898
2,6.018110e+01,-4360.247070,-6347.513184,12331.343750,-2366.590088,1946.592041,6636.523926,-3900.813965,-2533.625977
3,6.026490e+01,-4366.063965,-6371.688965,12311.220703,-2389.525879,1925.707031,6618.690918,-3921.460938,-2553.081055
4,6.032850e+01,-4366.397949,-6389.475098,12317.467773,-2385.092041,1935.291016,6630.850098,-3913.354980,-2544.020996
...,...,...,...,...,...,...,...,...,...
1201240,2.402589e+06,-4003.382080,-6123.448242,17756.750000,-2073.955078,2945.470947,6527.232910,-4694.318848,-2134.989990
1201241,2.402589e+06,-3984.547119,-6113.053223,17771.484375,-2069.998047,2958.916992,6539.487793,-4683.780762,-2116.774902
1201242,2.402589e+06,-3987.025879,-6137.084961,17768.527344,-2075.529053,2953.481934,6533.050781,-4688.500977,-2116.489014
1201243,2.402589e+06,-3993.750000,-6142.425781,17758.228516,-2079.010010,2941.322021,6519.651855,-4693.984863,-2135.514893


### 1.2 iMotions transform

In [ ]:
None

### 1.3 iMotions save -> Raw

In [ ]:
for data_config in IMOTIONS_LIST:
    output_dir = data_config.save_dir / participant.id
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    file_path = output_dir / f"{participant.id}_{data_config.name}.csv"
    participant_imotions.datasets[data_config.name].dataset.to_csv(
        file_path, 
        index=True)
    logging.info(f"Dataset '{data_config.name}' for participant {participant.id} saved to {file_path}")


[17:49:49] [INFO] [root] - Dataset 'trial' for participant 001_pilot_bjoern saved to /Users/visser/drive/PhD/Code/pain-measurement/data/_test/raw/001_pilot_bjoern/001_pilot_bjoern_trial.csv


[17:49:49] [INFO] [root] - Dataset 'temperature' for participant 001_pilot_bjoern saved to /Users/visser/drive/PhD/Code/pain-measurement/data/_test/raw/001_pilot_bjoern/001_pilot_bjoern_temperature.csv
[17:49:49] [INFO] [root] - Dataset 'rating' for participant 001_pilot_bjoern saved to /Users/visser/drive/PhD/Code/pain-measurement/data/_test/raw/001_pilot_bjoern/001_pilot_bjoern_rating.csv
[17:49:50] [INFO] [root] - Dataset 'eda' for participant 001_pilot_bjoern saved to /Users/visser/drive/PhD/Code/pain-measurement/data/_test/raw/001_pilot_bjoern/001_pilot_bjoern_eda.csv
[17:49:55] [INFO] [root] - Dataset 'ecg' for participant 001_pilot_bjoern saved to /Users/visser/drive/PhD/Code/pain-measurement/data/_test/raw/001_pilot_bjoern/001_pilot_bjoern_ecg.csv
[17:50:01] [INFO] [root] - Dataset 'eeg' for participant 001_pilot_bjoern saved to /Users/visser/drive/PhD/Code/pain-measurement/data/_test/raw/001_pilot_bjoern/001_pilot_bjoern_eeg.csv
[17:50:01] [INFO] [root] - Dataset 'pupillometry

## 2. Raw -> Trial

### 2.0 Raw config

In [ ]:
for data_config in RAW_LIST:
    data_config.load_dir = TEST_DIR / 'raw'
    data_config.save_dir = TEST_DIR / 'trial'


### 2.1 Raw load

In [ ]:
datasets = {}
for data_config in RAW_LIST:
    file_path = data_config.load_dir / participant.id / f"{participant.id}_{data_config.name}.csv"
    # Load and process data
    dataset = pd.read_csv(
        file_path,
        skiprows=None,
        usecols=lambda column: column in data_config.load_columns,
        )

    dataset = Data(name=data_config.name, dataset=dataset)
    datasets[data_config.name] = dataset

participant_raw = Participant(id=participant.id, datasets=datasets)
participant_raw.eeg


,Timestamp,EEG_RAW_Ch1,EEG_RAW_Ch2,EEG_RAW_Ch3,EEG_RAW_Ch4,EEG_RAW_Ch5,EEG_RAW_Ch6,EEG_RAW_Ch7,EEG_RAW_Ch8
0,5.982810e+01,-4361.342773,-6395.005859,12318.420898,-2381.229004,1936.005981,6634.855957,-3910.541992,-2543.639893
1,6.009020e+01,-4367.398926,-6375.790039,12322.760742,-2373.503906,1938.581055,6633.663086,-3906.250000,-2539.920898
2,6.018110e+01,-4360.247070,-6347.513184,12331.343750,-2366.590088,1946.592041,6636.523926,-3900.813965,-2533.625977
3,6.026490e+01,-4366.063965,-6371.688965,12311.220703,-2389.525879,1925.707031,6618.690918,-3921.460938,-2553.081055
4,6.032850e+01,-4366.397949,-6389.475098,12317.467773,-2385.092041,1935.291016,6630.850098,-3913.354980,-2544.020996
...,...,...,...,...,...,...,...,...,...
1201240,2.402589e+06,-4003.382080,-6123.448242,17756.750000,-2073.955078,2945.470947,6527.232910,-4694.318848,-2134.989990
1201241,2.402589e+06,-3984.547119,-6113.053223,17771.484375,-2069.998047,2958.916992,6539.487793,-4683.780762,-2116.774902
1201242,2.402589e+06,-3987.025879,-6137.084961,17768.527344,-2075.529053,2953.481934,6533.050781,-4688.500977,-2116.489014
1201243,2.402589e+06,-3993.750000,-6142.425781,17758.228516,-2079.010010,2941.322021,6519.651855,-4693.984863,-2135.514893


### 2.2 Raw transform

In [ ]:
# Special case for trial data: we need to add the stimuli seed column
# RawConfig datasets are missing the trial information (via Stimuli_Seed) and need to be merged with the trial data first
for data_config in RAW_LIST:
    pd.options.mode.chained_assignment = None  # default='warn'
    # add the stimuli seed column to all raw datasets of the participant
    if "Stimuli_Seed" not in participant_raw.datasets[data_config.name].dataset.columns:
        participant_raw.datasets[data_config.name].dataset = pd.merge(
            participant_raw.datasets[data_config.name].dataset, 
            participant_raw.trial, 
            on='Timestamp', how='outer')
        participant_raw.datasets[data_config.name].dataset.sort_values(by=['Timestamp'], inplace=True)
        participant_raw.datasets[data_config.name].dataset.reset_index(drop=True, inplace=True)
pd.options.mode.chained_assignment = 'warn'

for data_config in RAW_LIST:
    for transformation in data_config.transformations:
        participant_raw.datasets[data_config.name].dataset = transformation(participant_raw.datasets[data_config.name].dataset)
   
participant_raw.eeg

/Users/visser/drive/PhD/Code/pain-measurement/src/data/transform_data.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Stimuli_Seed'] = df['Stimuli_Seed'].astype(int)
/Users/visser/drive/PhD/Code/pain-measurement/src/data/transform_data.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Trial'] = df.Stimuli_Seed.diff().ne(0).cumsum()
/Users/visser/drive/PhD/Code/pain-measurement/src/data/transform_data.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

Timestamp  EEG_RAW_Ch1  EEG_RAW_Ch2  \
Trial Time                                                             
1     0 days 00:03:08.041000  1.880411e+05 -4107.475098 -6186.104004   
      0 days 00:03:08.055000  1.880554e+05 -4107.475098 -6186.104004   
      0 days 00:03:08.056000  1.880556e+05 -4117.774902 -6187.868164   
      0 days 00:03:08.057000  1.880566e+05 -4122.782227 -6187.009766   
      0 days 00:03:08.068000  1.880679e+05 -4110.289062 -6174.469238   
...                                    ...          ...          ...   
6     0 days 00:38:23.533000  2.303533e+06 -4180.479004 -6212.568848   
      0 days 00:38:23.565000  2.303565e+06 -4149.817871 -6184.912109   
      0 days 00:38:23.567000  2.303567e+06 -4155.587891 -6183.911133   
      0 days 00:38:23.568000  2.303568e+06 -4161.405762 -6189.823242   
      0 days 00:38:23.572000  2.303572e+06 -4161.405762 -6189.823242   

                               EEG_RAW_Ch3  EEG_RAW_Ch4  EEG_RAW_Ch5  \
Trial Time                                                             
1     0 days 00:03:08.041000  12735.891602 -2445.841064  2163.601074   
      0 days 00:03:08.055000  12735.891602 -2445.841064  2163.601074   
      0 days 00:03:08.056000  12731.649414 -2451.467041  2159.309082   
      0 days 00:03:08.057000  12735.654297 -2449.702881  2162.408936   
      0 days 00:03:08.068000  12745.142578 -2435.826904  2174.472900   
...                                    ...          ...          ...   
6     0 days 00:38:23.533000  17189.695312 -2142.954102  2721.452881   
      0 days 00:38:23.565000  17211.009766 -2132.271973  2750.444092   
      0 days 00:38:23.567000  17207.623047 -2130.413086  2746.343018   
      0 days 00:38:23.568000  17209.007812 -2123.165039  2743.339111   
      0 days 00:38:23.572000  17209.007812 -2123.165039  2743.339111   

                              EEG_RAW_Ch6  EEG_RAW_Ch7  EEG_RAW_Ch8  \
Trial Time                                                            
1     0 days 00:03:08.041000  6617.785156 -3973.197998 -2446.747070   
      0 days 00:03:08.055000  6617.785156 -3973.197998 -2446.747070   
      0 days 00:03:08.056000  6615.590820 -3977.537109 -2447.510010   
      0 days 00:03:08.057000  6617.165039 -3974.389893 -2446.604004   
      0 days 00:03:08.068000  6628.180176 -3959.273926 -2439.545898   
...                                   ...          ...          ...   
6     0 days 00:38:23.533000  6450.605957 -4731.226074 -2096.270996   
      0 days 00:38:23.565000  6468.105957 -4715.347168 -2078.770996   
      0 days 00:38:23.567000  6457.472168 -4726.838867 -2084.017090   
      0 days 00:38:23.568000  6464.004883 -4714.917969 -2084.969971   
      0 days 00:38:23.572000  6464.004883 -4714.917969 -2084.969971   

                              Stimuli_Seed  
Trial Time                                  
1     0 days 00:03:08.041000             1  
      0 days 00:03:08.055000             1  
      0 days 00:03:08.056000             1  
      0 days 00:03:08.057000             1  
      0 days 00:03:08.068000             1  
...                                    ...  
6     0 days 00:38:23.533000             6  
      0 days 00:38:23.565000             6  
      0 days 00:38:23.567000             6  
      0 days 00:38:23.568000             6  
      0 days 00:38:23.572000             6  

[177439 rows x 10 columns]

### 2.3 Raw save -> Trial

In [ ]:
for data_config in RAW_LIST:
    output_dir = data_config.save_dir / participant.id
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    file_path = output_dir / f"{participant.id}_{data_config.name}.csv"
    participant_imotions.datasets[data_config.name].dataset.to_csv(
        file_path, 
        index=True)
    logging.info(f"Dataset '{data_config.name}' for participant {participant.id} saved to {file_path}")


[17:50:05] [INFO] [root] - Dataset 'trial' for participant 001_pilot_bjoern saved to /Users/visser/drive/PhD/Code/pain-measurement/data/_test/trial/001_pilot_bjoern/001_pilot_bjoern_trial.csv


[17:50:05] [INFO] [root] - Dataset 'temperature' for participant 001_pilot_bjoern saved to /Users/visser/drive/PhD/Code/pain-measurement/data/_test/trial/001_pilot_bjoern/001_pilot_bjoern_temperature.csv
[17:50:05] [INFO] [root] - Dataset 'rating' for participant 001_pilot_bjoern saved to /Users/visser/drive/PhD/Code/pain-measurement/data/_test/trial/001_pilot_bjoern/001_pilot_bjoern_rating.csv
[17:50:06] [INFO] [root] - Dataset 'eda' for participant 001_pilot_bjoern saved to /Users/visser/drive/PhD/Code/pain-measurement/data/_test/trial/001_pilot_bjoern/001_pilot_bjoern_eda.csv
[17:50:11] [INFO] [root] - Dataset 'ecg' for participant 001_pilot_bjoern saved to /Users/visser/drive/PhD/Code/pain-measurement/data/_test/trial/001_pilot_bjoern/001_pilot_bjoern_ecg.csv
[17:50:17] [INFO] [root] - Dataset 'eeg' for participant 001_pilot_bjoern saved to /Users/visser/drive/PhD/Code/pain-measurement/data/_test/trial/001_pilot_bjoern/001_pilot_bjoern_eeg.csv
[17:50:18] [INFO] [root] - Dataset 'pu

Timestamp  EEG_RAW_Ch1  EEG_RAW_Ch2  \
Trial Time                                                             
1     0 days 00:03:08.041000  1.880411e+05 -4107.475098 -6186.104004   
      0 days 00:03:08.055000  1.880554e+05 -4107.475098 -6186.104004   
      0 days 00:03:08.056000  1.880556e+05 -4117.774902 -6187.868164   
      0 days 00:03:08.057000  1.880566e+05 -4122.782227 -6187.009766   
      0 days 00:03:08.068000  1.880679e+05 -4110.289062 -6174.469238   
...                                    ...          ...          ...   
6     0 days 00:38:23.533000  2.303533e+06 -4180.479004 -6212.568848   
      0 days 00:38:23.565000  2.303565e+06 -4149.817871 -6184.912109   
      0 days 00:38:23.567000  2.303567e+06 -4155.587891 -6183.911133   
      0 days 00:38:23.568000  2.303568e+06 -4161.405762 -6189.823242   
      0 days 00:38:23.572000  2.303572e+06 -4161.405762 -6189.823242   

                               EEG_RAW_Ch3  EEG_RAW_Ch4  EEG_RAW_Ch5  \
Trial Time                                                             
1     0 days 00:03:08.041000  12735.891602 -2445.841064  2163.601074   
      0 days 00:03:08.055000  12735.891602 -2445.841064  2163.601074   
      0 days 00:03:08.056000  12731.649414 -2451.467041  2159.309082   
      0 days 00:03:08.057000  12735.654297 -2449.702881  2162.408936   
      0 days 00:03:08.068000  12745.142578 -2435.826904  2174.472900   
...                                    ...          ...          ...   
6     0 days 00:38:23.533000  17189.695312 -2142.954102  2721.452881   
      0 days 00:38:23.565000  17211.009766 -2132.271973  2750.444092   
      0 days 00:38:23.567000  17207.623047 -2130.413086  2746.343018   
      0 days 00:38:23.568000  17209.007812 -2123.165039  2743.339111   
      0 days 00:38:23.572000  17209.007812 -2123.165039  2743.339111   

                              EEG_RAW_Ch6  EEG_RAW_Ch7  EEG_RAW_Ch8  \
Trial Time                                                            
1     0 days 00:03:08.041000  6617.785156 -3973.197998 -2446.747070   
      0 days 00:03:08.055000  6617.785156 -3973.197998 -2446.747070   
      0 days 00:03:08.056000  6615.590820 -3977.537109 -2447.510010   
      0 days 00:03:08.057000  6617.165039 -3974.389893 -2446.604004   
      0 days 00:03:08.068000  6628.180176 -3959.273926 -2439.545898   
...                                   ...          ...          ...   
6     0 days 00:38:23.533000  6450.605957 -4731.226074 -2096.270996   
      0 days 00:38:23.565000  6468.105957 -4715.347168 -2078.770996   
      0 days 00:38:23.567000  6457.472168 -4726.838867 -2084.017090   
      0 days 00:38:23.568000  6464.004883 -4714.917969 -2084.969971   
      0 days 00:38:23.572000  6464.004883 -4714.917969 -2084.969971   

                              Stimuli_Seed  
Trial Time                                  
1     0 days 00:03:08.041000             1  
      0 days 00:03:08.055000             1  
      0 days 00:03:08.056000             1  
      0 days 00:03:08.057000             1  
      0 days 00:03:08.068000             1  
...                                    ...  
6     0 days 00:38:23.533000             6  
      0 days 00:38:23.565000             6  
      0 days 00:38:23.567000             6  
      0 days 00:38:23.568000             6  
      0 days 00:38:23.572000             6  

[177439 rows x 10 columns]

In [ ]:
# Get dataframe without temperature
participant_raw.datasets['eeg'].dataset.drop(columns=['Temperature'])
participant_raw.temperature

KeyError: "['Temperature'] not found in axis"

In [ ]:
def merge_participant_datasets(participant: Participant) -> pd.DataFrame:
    data_frames = [data.dataset.drop(columns=['Timestamp','Stimuli_Seed']) for data in participant.datasets.values()]
    # Use reduce to merge all DataFrames on 'Timestamp'
    merged_df = reduce(
        # pd.concat would lead to duplicate timestamps
        lambda left, right: pd.merge(left, right, on='Time', how='outer'),
        data_frames
    )
    merged_df.sort_values(by=['Time'], inplace=True)
    logging.info(f"Dataframe shape: {merged_df.shape}")
    return merged_df


[23:54:38] [INFO] [root] - Dataframe shape: (596056, 67)


,Temperature,Rating,EDA_d_Battery,EDA_RAW,EDA_d_PacketReceptionRate,ECG_d_Battery,ECG_LL-RA,ECG_LA-RA,ECG_Vx-RL,ECG_LL-RA_HeartRate,...,Blink,BlinkRate,Pitch,Yaw,Roll,Interocular Distance,CPU Sys,Memory Sys,CPU Proc,Memory Proc
Time,,,,,,,,,,,,,,,,,,,,,
0 days 00:03:08.041000,38.600,48.083,3638.112821,6.057357,100.0,3756.082051,0.06289,-0.915007,-603.386639,66.349892,...,0.0,30.0,0.433156,1.623179,-0.515956,135.472092,37.263802,19.966390,23.124765,715.867188
0 days 00:03:08.046000,NaN,NaN,3638.112821,6.057357,100.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:03:08.047000,38.600,48.083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:03:08.049000,NaN,NaN,3659.958974,6.062463,100.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:03:08.050000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 00:38:23.562000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:38:23.565000,NaN,27.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:38:23.567000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def merge_participant_datasets_2(participant: Participant) -> pd.DataFrame:
    data_frames = [data.dataset.reset_index().drop(columns=['Time','Stimuli_Seed','Trial']) for data in participant.datasets.values()]
    # Use reduce to merge all DataFrames on 'Timestamp'
    merged_df = reduce(
        # pd.concat would lead to duplicate timestamps
        lambda left, right: pd.merge(left, right, on='Timestamp', how='outer'),
        data_frames
    )
    merged_df.sort_values(by=['Timestamp'], inplace=True)
    logging.info(f"Dataframe shape: {merged_df.shape}")
    return merged_df

In [ ]:
# %%timeit
merge_participant_datasets(participant_raw)


[00:00:05] [INFO] [root] - Dataframe shape: (596056, 67)


,Temperature,Rating,EDA_d_Battery,EDA_RAW,EDA_d_PacketReceptionRate,ECG_d_Battery,ECG_LL-RA,ECG_LA-RA,ECG_Vx-RL,ECG_LL-RA_HeartRate,...,Blink,BlinkRate,Pitch,Yaw,Roll,Interocular Distance,CPU Sys,Memory Sys,CPU Proc,Memory Proc
Time,,,,,,,,,,,,,,,,,,,,,
0 days 00:03:08.041000,38.600,48.083,3638.112821,6.057357,100.0,3756.082051,0.06289,-0.915007,-603.386639,66.349892,...,0.0,30.0,0.433156,1.623179,-0.515956,135.472092,37.263802,19.966390,23.124765,715.867188
0 days 00:03:08.046000,NaN,NaN,3638.112821,6.057357,100.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:03:08.047000,38.600,48.083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:03:08.049000,NaN,NaN,3659.958974,6.062463,100.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:03:08.050000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 00:38:23.562000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:38:23.565000,NaN,27.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:38:23.567000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# %%timeit
merge_participant_datasets_2(participant_raw)


[00:00:07] [INFO] [root] - Dataframe shape: (660553, 68)


,Timestamp,Temperature,Rating,EDA_d_Battery,EDA_RAW,EDA_d_PacketReceptionRate,ECG_d_Battery,ECG_LL-RA,ECG_LA-RA,ECG_Vx-RL,...,Blink,BlinkRate,Pitch,Yaw,Roll,Interocular Distance,CPU Sys,Memory Sys,CPU Proc,Memory Proc
0,1.880411e+05,38.600,48.083,3638.112821,6.057357,100.0,3756.082051,0.06289,-0.915007,-603.386639,...,0.0,30.0,0.433156,1.623179,-0.515956,135.472092,37.263802,19.966390,23.124765,715.867188
107446,1.880461e+05,NaN,NaN,3638.112821,6.057357,100.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,1.880471e+05,38.600,48.083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107447,1.880491e+05,NaN,NaN,3659.958974,6.062463,100.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
517945,1.880501e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107445,2.303565e+06,NaN,27.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
517942,2.303565e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
517943,2.303567e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
517944,2.303568e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Trial -> Processed

### 3.1 Trial load

### 3.2 Trial transform

### 3.3 Trial save

In [ ]:
obj = pd.Series([4, 7, -5, 3])
a = obj[obj > 0]
a, a.reset_index(drop=True)

(0    4
 1    7
 3    3
 dtype: int64,
 0    4
 1    7
 2    3
 dtype: int64)

In [ ]:
animals = [
    'cat',
    'dog',
    'panda'
]
animals[0] = 'bug'
new_animals = animals
new_animals[0] = 'elephant'

print(animals)
print(new_animals)

['elephant', 'dog', 'panda']
['elephant', 'dog', 'panda']


In [ ]:
a = [1, 2, 3, 4]
b = a
b[1] = 0
a,b

([1, 0, 3, 4], [1, 0, 3, 4])

In [ ]:
points = np.arange(-5, 5, 0.01) # 100 equally spaced points

xs, ys = np.meshgrid(points, points)

ys.T


array([[-5.  , -4.99, -4.98, ...,  4.97,  4.98,  4.99],
       [-5.  , -4.99, -4.98, ...,  4.97,  4.98,  4.99],
       [-5.  , -4.99, -4.98, ...,  4.97,  4.98,  4.99],
       ...,
       [-5.  , -4.99, -4.98, ...,  4.97,  4.98,  4.99],
       [-5.  , -4.99, -4.98, ...,  4.97,  4.98,  4.99],
       [-5.  , -4.99, -4.98, ...,  4.97,  4.98,  4.99]])

In [ ]:
arr = np.array([0, 1, 2, 3])
arr.cumsum()


array([0, 1, 3, 6])

In [ ]:
xs

array([[-5.  , -4.99, -4.98, ...,  4.97,  4.98,  4.99],
       [-5.  , -4.99, -4.98, ...,  4.97,  4.98,  4.99],
       [-5.  , -4.99, -4.98, ...,  4.97,  4.98,  4.99],
       ...,
       [-5.  , -4.99, -4.98, ...,  4.97,  4.98,  4.99],
       [-5.  , -4.99, -4.98, ...,  4.97,  4.98,  4.99],
       [-5.  , -4.99, -4.98, ...,  4.97,  4.98,  4.99]])

In [ ]:
participant = load_participant_datasets(PARTICIPANT_LIST[0], RAW_LIST)

UnboundLocalError: cannot access local variable 'file_start_index' where it is not associated with a value